# Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import json
from glob import glob

import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models
!pip install pytorch-lightning
import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.utilities.seed import seed_everything
!pip install albumentations==0.4.6
import albumentations as A
from albumentations.pytorch import ToTensorV2

from PIL import Image
!pip install timm
import timm
!pip install wandb
import wandb

import torch.nn            as nn
import torch.optim         as optim
import torch.utils.data    as data
import torch.nn.functional as F
# !pip install model
!pip install https://github.com/ufoym/imbalanced-dataset-sampler/archive/master.zip
# from model import LSTMfcn
from torchsampler import ImbalancedDatasetSampler

ROOT_DIR = '/content/drive/MyDrive/crop_disease2/'
torch.set_default_dtype(torch.float32)

     |████████████████████████████████| 527 kB 11.5 MB/s 
     |████████████████████████████████| 829 kB 34.5 MB/s 
     |████████████████████████████████| 396 kB 71.4 MB/s 
     |████████████████████████████████| 133 kB 62.1 MB/s 
     |████████████████████████████████| 952 kB 85.6 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 1.1 MB 65.3 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 79.7 MB/s 
     |████████████████████████████████| 271 kB 68.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=96f465c9e6b8b8bb9697e8a4362fe608e7a6cb129123fe6f58b284c27adc671a
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfu

     |████████████████████████████████| 117 kB 14.7 MB/s 
     |████████████████████████████████| 948 kB 80.8 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=476891fc4b6c19347214f995205c7f1ac39991fa1256d7a6be66ccf6d69367f0
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12
     |████████████████████████████████| 431 kB 13.9 MB/s 
     |████████████████████████████████| 1.7 MB 14.2 MB/s 
     |████████████████████████████████| 143 kB 82.2 MB/s 
     |████████████████████████████████| 180 kB 71.9 MB/s 
     |

In [3]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# Initialize

In [4]:
def initialize():
    csv_feature_dict = { 
        '내부 온도 1 평균': [3.4, 47.3],
        '내부 온도 1 최고': [3.4, 47.6],
        '내부 온도 1 최저': [3.3, 47.0],
        '내부 습도 1 평균': [0.0, 100.0],
        '내부 습도 1 최고': [0.0, 100.0],
        '내부 습도 1 최저': [0.0, 100.0],

        '내부 이슬점 평균': [0.2, 34.5],
        '내부 이슬점 최고': [0.1, 34.7],
        '내부 이슬점 최저': [0.0, 34.4],
    }

    crop = {'1': '딸기', '2': '토마토', '3': '파프리카', '4': '오이', '5': '고추', '6': '시설포도'}
    disease = {
        '1': {
            'a1': '딸기잿빛곰팡이병', 'a2': '딸기흰가루병', 'b1': '냉해피해', 
            'b6': '다량원소결핍 (N)', 'b7': '다량원소결핍 (P)', 'b8': '다량원소결핍 (K)'
        },
        '2': {
            'a5': '토마토흰가루병', 'a6': '토마토잿빛곰팡이병', 'b2': '열과', 'b3': '칼슘결핍',
            'b6': '다량원소결핍 (N)', 'b7': '다량원소결핍 (P)', 'b8': '다량원소결핍 (K)'
        },
        '3': {
            'a9': '파프리카흰가루병', 'a10': '파프리카잘록병', 'b3': '칼슘결핍', 
            'b6': '다량원소결핍 (N)', 'b7': '다량원소결핍 (P)', 'b8': '다량원소결핍 (K)'
        },
        '4': {
            'a3': '오이노균병', 'a4': '오이흰가루병', 'b1': '냉해피해', 
            'b6': '다량원소결핍 (N)', 'b7': '다량원소결핍 (P)', 'b8': '다량원소결핍 (K)' 
        },
        '5': {
            'a7': '고추탄저병', 'a8': '고추흰가루병', 'b3': '칼슘결핍', 
            'b6': '다량원소결핍 (N)', 'b7': '다량원소결핍 (P)', 'b8': '다량원소결핍 (K)'
        },
        '6': {'a11': '시설포도탄저병', 'a12': '시설포도노균병', 'b4': '일소피해', 'b5': '축과병'}
    }
    risk = {'1': '초기', '2': '중기', '3': '말기'}
    
    label_description = {}
    for key, value in disease.items():
        label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
        for disease_code in value:
            for risk_code in risk:
                label = f'{key}_{disease_code}_{risk_code}'
                label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'
                
    label_encoder = {key: idx for idx, key in enumerate(label_description)}
    label_decoder = {val: key for key, val in label_encoder.items()}
    
    return csv_feature_dict, label_encoder, label_decoder


def split_data(split_rate=0.2, seed=42, mode='train'):
    """
    Use for model trained image and time series.
    """
    if mode == 'train':
        train = sorted(glob(f'{ROOT_DIR}/train/*'))
        
        labelsss = pd.read_csv(f'{ROOT_DIR}/train.csv')['label']
        train, val = train_test_split(
            train, test_size=split_rate, random_state=seed, stratify=labelsss)
        
        return train, val
    elif mode == 'test':
        test = sorted(glob(f'{ROOT_DIR}/test/*'))

        return test
    
    
csv_feature_dict, label_encoder, label_decoder = initialize()

# Configuration

In [5]:
SEED = 42
IMAGE_WIDTH = 528
IMAGE_HEIGHT = 528
BATCH_SIZE = 8
CLASS_N = len(label_encoder)
LEARNING_RATE = 5e-4#5e-4
NUM_FEATURES = len(csv_feature_dict)
MAX_LEN = 590#24*6
DROPOUT_RATE = 0.1
EPOCHS = 300
NUM_WORKERS = 16

# Dataset

In [6]:
torch.multiprocessing.set_sharing_strategy('file_system')


class CustomDataset(Dataset):
    def __init__(
        self, 
        files, 
        csv_feature_dict, 
        label_encoder,
        labels=None,
        transforms=None,
        mode='train',
    ):
        self.mode = mode
        self.files = files
        
        self.csv_feature_dict = csv_feature_dict
        
        if files is not None:
            self.csv_feature_check = [0]*len(self.files)
            self.csv_features = [None]*len(self.files)
            
        self.max_len = MAX_LEN
        self.num_features = NUM_FEATURES
        
        self.label_encoder = label_encoder
        
        self.transforms = transforms

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split(os.sep)[-1]
        
        # csv
        csv_path = f'{file}/{file_name}.csv'
        df = pd.read_csv(csv_path)[self.csv_feature_dict.keys()]
        df = df.replace('-', 0)

        # MinMax scaling
        for col in df.columns:
            df[col] = df[col].astype(float) - self.csv_feature_dict[col][0]
            df[col] = df[col] / (self.csv_feature_dict[col][1]-self.csv_feature_dict[col][0])

        df_dict = {}
        df_dict['내부 온도 1 최저'] = np.max(df[['내부 온도 1 최저']].to_numpy(), -1)
        df_dict['내부 온도 1 평균'] = np.max(df[['내부 온도 1 평균']].to_numpy(), -1)
        df_dict['내부 온도 1 최고'] = np.max(df[['내부 온도 1 최고']].to_numpy(), -1)
        df_dict['내부 습도 1 최저'] = np.max(df[['내부 습도 1 최저']].to_numpy(), -1)
        df_dict['내부 습도 1 평균'] = np.max(df[['내부 습도 1 평균']].to_numpy(), -1)
        df_dict['내부 습도 1 최고'] = np.max(df[['내부 습도 1 최고']].to_numpy(), -1)
        df_dict['내부 이슬점 최저'] = np.max(df[['내부 이슬점 최저']].to_numpy(), -1)
        df_dict['내부 이슬점 평균'] = np.max(df[['내부 이슬점 평균']].to_numpy(), -1)
        df_dict['내부 이슬점 최고'] = np.max(df[['내부 이슬점 최고']].to_numpy(), -1)
        #df_dict['내부 CO2 최저'] = np.max(df[['내부 CO2 최저']].to_numpy(), -1)
        #df_dict['내부 CO2 평균'] = np.max(df[['내부 CO2 평균']].to_numpy(), -1)
        #df_dict['내부 CO2 최고'] = np.max(df[['내부 CO2 최고']].to_numpy(), -1)
        #df_dict['외부 누적일사 평균'] = np.max(df[['외부 누적일사 평균']].to_numpy(), -1)
        seq_len = len(df_dict['내부 온도 1 최저'])
        df = pd.DataFrame(df_dict)

        after_df = pd.DataFrame(df_dict).to_numpy()
        x        = np.zeros([self.max_len, self.num_features])

        h, w      = after_df.shape
        x[0:h, :] = after_df
        
        # image
        image_path = f'{file}/{file_name}.jpg'
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        h,w,c = img.shape
        if w < h: # width padding
            pad_w = (h - w) // 2
            pad_img = np.zeros([h,h,3]).astype(np.uint8)
            pad_img[:,pad_w:pad_w+w] = img
            img = pad_img
        elif w > h: # height padding
            pad_h = (w - h) // 2
            pad_img = np.zeros([w,w,3]).astype(np.uint8)
            pad_img[pad_h:pad_h+h,:] = img
            img = pad_img
        
        if self.transforms is not None:
            img = self.transforms(image=img)['image']
            #img = self.transforms(Image.fromarray(img)).float()
        
        if self.mode == 'train':
            json_path = f'{file}/{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = f'{crop}_{disease}_{risk}'
            
            return {
                'img': img,
                'csv_feature': torch.tensor(x, dtype=torch.float32),
                'seq_len' : seq_len,
                'label': torch.tensor(self.label_encoder[label], dtype=torch.long)
            }
        else:
            return {
                'img': img,
                'seq_len' : seq_len,
                'csv_feature': torch.tensor(x, dtype=torch.float32)
            }
    def get_labels(self):
        labels = []
        for file in self.files:
            file_name = file.split(os.sep)[-1]
            json_path = f'{file}/{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = f'{crop}_{disease}_{risk}'
            labels.append(label)
        return labels
    
class CustomDataModule(LightningDataModule):
    def __init__(
        self,
        train=None,
        val=None,
        test=None,
        csv_feature_dict=None,
        label_encoder=None,
        train_transforms=None,
        val_transforms=None,
        predict_transforms=None,
        num_workers=32,
        batch_size=8,
    ):
        super().__init__()
        self.train = train
        self.val = val
        self.test = test
        self.csv_feature_dict = csv_feature_dict
        self.label_encoder = label_encoder
        assert self.csv_feature_dict is not None
        assert self.label_encoder is not None
        self.train_transforms = train_transforms
        self.val_transforms = val_transforms
        self.predict_transforms = predict_transforms
        self.num_workers = num_workers
        self.batch_size = batch_size

    def setup(self, stage=None):
        self.train_dataset = CustomDataset(
            self.train, 
            self.csv_feature_dict,
            self.label_encoder,
            transforms=self.train_transforms,
        )
        self.valid_dataset = CustomDataset(
            self.val, 
            self.csv_feature_dict,
            self.label_encoder,
            transforms=self.train_transforms,
        )
        self.predict_dataset = CustomDataset(
            self.test, 
            self.csv_feature_dict,
            self.label_encoder,
            transforms=self.predict_transforms,
            mode='test'
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            pin_memory=True,
            shuffle=False,
            num_workers=self.num_workers,
            sampler=ImbalancedDatasetSampler(self.train_dataset),
        )

    def val_dataloader(self):
        return DataLoader(
            self.valid_dataset,
            batch_size=self.batch_size,
            pin_memory=True,
            shuffle=False,
            num_workers=self.num_workers,
        )
        
    def predict_dataloader(self):
        return DataLoader(
            self.predict_dataset,
            batch_size=self.batch_size,
            pin_memory=True,
            shuffle=False,
            num_workers=self.num_workers,
        )


# Model

In [7]:
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [8]:
tar = []
out = []

def accuracy_function(real, pred):    
    global tar, out
    real = real.cpu()
    pred = torch.argmax(pred, dim=1).cpu()
    tar.append(real)
    out.append(pred)
    score = f1_score(real, pred, average='macro')
    return score


class CNN_Encoder(nn.Module):
    def __init__(self, class_n, rate=0.1):
        super(CNN_Encoder, self).__init__()
        #self.model = models.resnet50(pretrained=True)
        self.model = timm.create_model('tf_efficientnet_b4_ns', pretrained=True)
        # self.model = timm.create_model('tf_efficientnet_b6_ns', pretrained=True)#, num_classes=class_n)
        #self.model = timm.create_model('nfnet_f4', pretrained=True)
        #self.model, self.preprocess = clip.load("ViT-B/32", device=device)
        self.model = self.model.float()
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output


    
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class MLSTMfcn(nn.Module):
    def __init__(self, *, num_classes, max_seq_len, num_features,
                 num_lstm_out=128, num_lstm_layers=1, 
                 conv1_nf=128, conv2_nf=256, conv3_nf=128,
                 lstm_drop_p=0.8, fc_drop_p=0.3):
        super(MLSTMfcn, self).__init__()
        self.num_classes = num_classes
        self.max_seq_len = max_seq_len
        self.num_features = num_features

        self.num_lstm_out = num_lstm_out
        self.num_lstm_layers = num_lstm_layers

        self.conv1_nf = conv1_nf
        self.conv2_nf = conv2_nf
        self.conv3_nf = conv3_nf

        self.lstm_drop_p = lstm_drop_p
        self.fc_drop_p = fc_drop_p

        self.lstm = nn.LSTM(input_size=self.num_features, 
                            hidden_size=self.num_lstm_out,
                            num_layers=self.num_lstm_layers,
                            batch_first=True)
        
        self.conv1 = nn.Conv1d(self.num_features, self.conv1_nf, 8)
        self.conv2 = nn.Conv1d(self.conv1_nf, self.conv2_nf, 5)
        self.conv3 = nn.Conv1d(self.conv2_nf, self.conv3_nf, 3)

        self.bn1 = nn.BatchNorm1d(self.conv1_nf)
        self.bn2 = nn.BatchNorm1d(self.conv2_nf)
        self.bn3 = nn.BatchNorm1d(self.conv3_nf)

        self.se1 = SELayer(self.conv1_nf)  # ex 128
        self.se2 = SELayer(self.conv2_nf)  # ex 256

        self.relu = nn.ReLU()
        self.lstmDrop = nn.Dropout(self.lstm_drop_p)
        self.convDrop = nn.Dropout(self.fc_drop_p)

        self.fc = nn.Linear(self.conv3_nf+self.num_lstm_out, 128)

        self.out_layer = nn.Linear(1000+128, self.num_classes)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, enc_out, x, seq_lens):
        ''' input x should be in size [B,T,F], where 
            B = Batch size
            T = Time samples
            F = features
        '''
        x1 = nn.utils.rnn.pack_padded_sequence(x, seq_lens.cpu(), 
                                               batch_first=True, 
                                               enforce_sorted=False)
        x1, (ht,ct) = self.lstm(x1)
        x1, _ = nn.utils.rnn.pad_packed_sequence(x1, batch_first=True, 
                                                 padding_value=0.0)
        x1 = x1[:,-1,:]
        
        x2 = x.transpose(2,1)
        x2 = self.convDrop(self.relu(self.bn1(self.conv1(x2))))
        x2 = self.se1(x2)
        x2 = self.convDrop(self.relu(self.bn2(self.conv2(x2))))
        x2 = self.se2(x2)
        x2 = self.convDrop(self.relu(self.bn3(self.conv3(x2))))
        x2 = torch.mean(x2,2)
        
        x_all = torch.cat((x1,x2),dim=1)
        x_out = self.fc(x_all)
        concat = torch.cat([enc_out, x_out], dim=1)  # enc_out + hidden 
        output = self.dropout(concat)
        x_output = self.out_layer(output)
        x_out = F.log_softmax(x_output, dim=1)

        return x_out

class BaseModel(LightningModule):
    def __init__(
        self,
        cnn,
        rnn,
        criterion,
        learning_rate=5e-4,
    ):
        super(BaseModel, self).__init__()
        global NUM_FEATURES
        
        self.cnn = cnn
        self.rnn = rnn
        self.learning_rate = learning_rate
        self.criterion = criterion
        
    def configure_optimizers(self):
        self.optimizer = optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=5e-2)
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=92 * 300 * 1.2) # epoch 25 step 92
        return self.optimizer

    def forward(self, img, seq, seq_len):
        cnn_output = self.cnn(img)
        output = self.rnn(cnn_output, seq, seq_len)
        
        return output

    def training_step(self, batch, batch_idx):
        img = batch['img']
        csv_feature = batch['csv_feature']
        label = batch['label']
        seq_len = batch['seq_len']
        
        output = self(img, csv_feature, seq_len)
        loss = self.criterion(output, label)
        score = accuracy_function(label, output)
        
        self.log('train_loss', loss, prog_bar=True, logger=True)
        self.log('train_score', score, prog_bar=True, logger=True)
        # wandb.log({'train score': score,'train loss': loss})
        
        self.scheduler.step()
        return {'loss': loss, 'train_score': score}        

    def validation_step(self, batch, batch_idx):
        img = batch['img']
        csv_feature = batch['csv_feature']
        label = batch['label']
        seq_len = batch['seq_len']
        
        output = self(img, csv_feature, seq_len)
        loss = self.criterion(output, label)
        score = accuracy_function(label, output)
        
        self.log('val_loss', loss, prog_bar=True, logger=True)
        self.log('val_score', score, prog_bar=True, logger=True)
        # wandb.log({'val score': score,'val loss': loss})
        
        return {'val_loss': loss, 'val_score': score}
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        img = batch['img']
        seq = batch['csv_feature']
        seq_len = batch['seq_len']
        
        output = self(img, seq, seq_len)
        output = torch.argmax(output, dim=1)
        
        return output

    def validation_epoch_end(self, validation_step_outputs):
        global tar, out
        tar = torch.cat(tar)
        out = torch.cat(out)

        # wandb.log({"Confusion Matrix1" :wandb.plot.confusion_matrix(preds=out.cpu().numpy(), y_true=tar.cpu().numpy())})

        tar = []
        out = []

# from src import cct_14_7x2_384_fl

class CNN2RNNModel(BaseModel):
    def __init__(
        self,
        max_len, 
        num_features, 
        class_n,
        rate=0.1,
        learning_rate=5e-4,
    ):
        cnn = CNN_Encoder(class_n)#cct_14_7x2_384_fl(pretrained=True, progress=True)#CNN_Encoder(class_n)
        rnn = MLSTMfcn(num_classes=class_n, max_seq_len=max_len, num_features=num_features)

        criterion = nn.NLLLoss()#nn.CrossEntropyLoss()
        
        super(CNN2RNNModel, self).__init__(cnn, rnn, criterion, learning_rate)

# Train

In [9]:
def get_train_transforms(height, width):
    return A.Compose([
        A.Resize(height=height, width=width, interpolation=cv2.INTER_CUBIC),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(25, interpolation=cv2.INTER_CUBIC),
        A.Normalize([0.4419, 0.5274, 0.3488], [0.2152, 0.1902, 0.2384]),
        ToTensorV2(),
    ])

def get_valid_transforms(height, width):
    return A.Compose([
        A.Resize(height=height, width=width, interpolation=cv2.INTER_CUBIC),
        A.Normalize([0.4419, 0.5274, 0.3488], [0.2152, 0.1902, 0.2384]),
        ToTensorV2()
    ])


def train(model_name, csv_feature_dict, label_encoder, seed=42):
    """
    Use for model trained image and time series.
    """
    train_data, val_data = split_data(seed=seed, mode='train')
    
    train_transforms = get_train_transforms(IMAGE_HEIGHT, IMAGE_WIDTH)
    val_transforms = get_valid_transforms(IMAGE_HEIGHT, IMAGE_WIDTH)
    
    data_module = CustomDataModule(
        train=train_data,
        val=val_data,
        csv_feature_dict=csv_feature_dict,
        label_encoder=label_encoder,
        train_transforms=train_transforms,
        val_transforms=val_transforms,
        num_workers=NUM_WORKERS,
        batch_size=BATCH_SIZE,
    )
    
    model = CNN2RNNModel(
        max_len=MAX_LEN, 
        num_features=NUM_FEATURES, 
        class_n=CLASS_N, 
        rate=DROPOUT_RATE,
        learning_rate=LEARNING_RATE,
    )
    
    ckpt_path = f'/content/drive/MyDrive/crop_disease2/weights/{model_name}/'
    if not os.path.exists(ckpt_path):
        os.makedirs(ckpt_path)
    
    checkpoint = ModelCheckpoint(
        monitor='val_score',
        dirpath=ckpt_path,
        filename='{epoch}-{val_score:.3f}',
        save_top_k=-1,
        mode='max',
        save_weights_only=True,
    )

    trainer = pl.Trainer(
        max_epochs=EPOCHS,
        gpus=1,
        precision=16,
        callbacks=[checkpoint],
        log_every_n_steps=5,
    )

    trainer.fit(model, data_module)

# Eval

In [10]:
def get_predict_transforms(height, width):
    return A.Compose([
        A.Resize(height=height, width=width, interpolation=cv2.INTER_CUBIC),
        A.Normalize([0.4419, 0.5274, 0.3488], [0.2152, 0.1902, 0.2384]),
        ToTensorV2(),
    ])


def get_submission(outputs, save_dir, save_filename, label_decoder):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        
    outputs = [o.detach().cpu().numpy() for batch in outputs
                                        for o in batch]
    preds = np.array([label_decoder[int(val)] for val in outputs])
    
    submission = pd.read_csv(f'{ROOT_DIR}/sample_submission.csv')
    submission['label'] = preds
    
    save_file_path = os.path.join(save_dir, save_filename)
    
    submission.to_csv(save_file_path, index=False)


def eval(
    ckpt_path, 
    csv_feature_dict, 
    label_encoder, 
    label_decoder,
    submit_save_dir='/content/drive/MyDrive/crop_disease2/submission',
    submit_save_name='high_score.csv',
):
    test_data = split_data(mode='test')
    
    predict_transforms = get_predict_transforms(IMAGE_HEIGHT, IMAGE_WIDTH)
    
    data_module = CustomDataModule(
        test=test_data,
        csv_feature_dict=csv_feature_dict,
        label_encoder=label_encoder,
        predict_transforms=predict_transforms,
        num_workers=NUM_WORKERS,
        batch_size=BATCH_SIZE,
    )
    
    model = CNN2RNNModel(
        max_len=590,  
        num_features=len(csv_feature_dict), 
        class_n=len(label_encoder),
    )

    trainer = pl.Trainer(
        gpus=1,
        precision=16,
    )

    ckpt = torch.load(ckpt_path)
    model.load_state_dict(ckpt['state_dict'])

    outputs = trainer.predict(model, data_module)

    get_submission(outputs, submit_save_dir, submit_save_name, label_decoder)

# Main

In [ ]:
seed_everything(SEED)

MODEL_NAME = 'tf_efficientnet_b4_ns'

train(MODEL_NAME, csv_feature_dict, label_encoder, seed=SEED)

Global seed set to 42
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:122: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:141: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  "DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7."
Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b4_ns-d6313a46.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b4_ns-d6313a46.pth
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/d

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Global seed set to 42
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# CKPT_PATH = '/content/drive/MyDrive/crop_disease2/weights/resnet51/epoch=59-val_score=0.953.ckpt'

# eval(CKPT_PATH, csv_feature_dict, label_encoder, label_decoder)    